## This notebook explores the original 2536 records for non standard remarks after removing the duplicated remarks.
### The output of this notebook is:
#### 1. 479 records with runway headings
#### 2. 2057 records sent to Lex 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import geopy
from geopy.distance import geodesic
import re

In [2]:
# Read in the csv file that has all remaining non-standard remarks
non_std_full = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/02-Incidents_Cleaned_NonStandard.csv')
non_std_full 

,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,REMARKS,MORID.CEDAR,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
0,1/4/2018,5:00,2018-01-04T05:00Z,ASA918,DEN,UAS,NaN,WOC,W-ROC reports aircraft reported a white UAS at...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/4/2018,8:31,2018-01-04T08:31Z,REH9,DEN,UAS,NaN,SAN,Reported a red and white UAS passed helicopter...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1/4/2018,10:50,2018-01-04T10:50Z,NaN,DEN,UAS,NaN,SHV,ATCT personnel reported observing a large UAS ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1/5/2018,11:35,2018-01-05T11:35Z,SKW3019,DEN,UAS,NaN,TUS,"Aircrew reported observing a light from a UAS,...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/6/2018,1:56,2018-01-06T01:56Z,N1671C,DEN,UAS,NaN,IWA,Aircraft reported UAS activity while entering ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3416,8/23/2021,18:34,2021-08-23T18:34Z,NaN,JATOC,UAS,NaN,SAT,"1441 EDT / 1841 UTC 8/23/2021\r\nHAVOC39, C-1...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3417,8/23/2021,21:53,2021-08-23T21:53Z,NaN,JATOC,UAS,NaN,ORD,1756 EDT / 2156 UTC 8/23/2021\r\nChicago OHare...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3418,8/25/2021,15:35,2021-08-25T15:35Z,SIL9006,DEN,UAS,NaN,TPA,Aircraft observed two UAS off the left side wh...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3419,8/26/2021,0:47,2021-08-26T00:47Z,NaN,DEN,UAS,NaN,BOS,MA State PD observed a UAS over the airport at...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Read in the csv file that has the 885 records that were hit by initial regex
non_std_loc = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/non-std_format_uas_loc.csv')
non_std_loc1 = non_std_loc['REMARKS']
non_std_loc1

0      Aircraft reported UAS activity while entering ...
1      Reported a UAS off the left side at 500 feet w...
2      Reported a black UAS off the left side of the ...
3      Aircraft reported UAS pass below aircraft whil...
4      Aircraft reported 3 large UAS at 3,000 feet 3....
                             ...                        
880    Facility manager observed a UAS from the groun...
881    NWS Seal Beach Base personnel observed a UAS o...
882    PIC reports aircraft struck a multi-colored, 2...
883    2120 EDT / 0120 UTC 08/23/2021\r\nENY3961 advi...
884    Aircraft observed two UAS off the left side wh...
Name: REMARKS, Length: 885, dtype: object

In [4]:
# Merge both datasets to find the unmatched later
non_match = pd.merge( non_std_loc1, non_std_full, how='outer',indicator=True, on ='REMARKS')
non_match

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,_merge
0,Aircraft reported UAS activity while entering ...,1/6/2018,1:56,2018-01-06T01:56Z,N1671C,DEN,UAS,NaN,IWA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,Reported a UAS off the left side at 500 feet w...,1/12/2018,3:54,2018-01-12T03:54Z,N604Z,DEN,UAS,NaN,SW1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,Reported a black UAS off the left side of the ...,1/12/2018,5:58,2018-01-12T05:58Z,SKW3398,DEN,UAS,NaN,SAN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,Aircraft reported UAS pass below aircraft whil...,1/15/2018,5:30,2018-01-15T05:30Z,UPS3746,DEN,UAS,NaN,OAK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,"Aircraft reported 3 large UAS at 3,000 feet 3....",1/21/2018,8:01,2018-01-21T08:01Z,N7882C,DEN,UAS,NaN,LAS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3416,"1335 EDT / 1735 UTC 8/23/2021\r\nHAVOC39, C130...",8/23/2021,17:30,2021-08-23T17:30Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
3417,"1441 EDT / 1841 UTC 8/23/2021\r\nHAVOC39, C-1...",8/23/2021,18:34,2021-08-23T18:34Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
3418,1756 EDT / 2156 UTC 8/23/2021\r\nChicago OHare...,8/23/2021,21:53,2021-08-23T21:53Z,NaN,JATOC,UAS,NaN,ORD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
3419,MA State PD observed a UAS over the airport at...,8/26/2021,0:47,2021-08-26T00:47Z,NaN,DEN,UAS,NaN,BOS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only


In [5]:
# remove the mathcing records. We have 2536 remaining records
non_match = non_match[(non_match._merge != 'both')]
non_match

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,_merge
885,W-ROC reports aircraft reported a white UAS at...,1/4/2018,5:00,2018-01-04T05:00Z,ASA918,DEN,UAS,NaN,WOC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
886,Reported a red and white UAS passed helicopter...,1/4/2018,8:31,2018-01-04T08:31Z,REH9,DEN,UAS,NaN,SAN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
887,ATCT personnel reported observing a large UAS ...,1/4/2018,10:50,2018-01-04T10:50Z,NaN,DEN,UAS,NaN,SHV,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
888,"Aircrew reported observing a light from a UAS,...",1/5/2018,11:35,2018-01-05T11:35Z,SKW3019,DEN,UAS,NaN,TUS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
889,Reported observing a UAS operating over downto...,1/6/2018,9:13,2018-01-06T09:13Z,N31197,DEN,UAS,NaN,TUL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3416,"1335 EDT / 1735 UTC 8/23/2021\r\nHAVOC39, C130...",8/23/2021,17:30,2021-08-23T17:30Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
3417,"1441 EDT / 1841 UTC 8/23/2021\r\nHAVOC39, C-1...",8/23/2021,18:34,2021-08-23T18:34Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
3418,1756 EDT / 2156 UTC 8/23/2021\r\nChicago OHare...,8/23/2021,21:53,2021-08-23T21:53Z,NaN,JATOC,UAS,NaN,ORD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
3419,MA State PD observed a UAS over the airport at...,8/26/2021,0:47,2021-08-26T00:47Z,NaN,DEN,UAS,NaN,BOS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only


In [6]:
# Drop unnecessary columns
non_match = non_match.drop(['_merge'], axis=1)
non_match

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
885,W-ROC reports aircraft reported a white UAS at...,1/4/2018,5:00,2018-01-04T05:00Z,ASA918,DEN,UAS,NaN,WOC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886,Reported a red and white UAS passed helicopter...,1/4/2018,8:31,2018-01-04T08:31Z,REH9,DEN,UAS,NaN,SAN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
887,ATCT personnel reported observing a large UAS ...,1/4/2018,10:50,2018-01-04T10:50Z,NaN,DEN,UAS,NaN,SHV,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
888,"Aircrew reported observing a light from a UAS,...",1/5/2018,11:35,2018-01-05T11:35Z,SKW3019,DEN,UAS,NaN,TUS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
889,Reported observing a UAS operating over downto...,1/6/2018,9:13,2018-01-06T09:13Z,N31197,DEN,UAS,NaN,TUL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3416,"1335 EDT / 1735 UTC 8/23/2021\r\nHAVOC39, C130...",8/23/2021,17:30,2021-08-23T17:30Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3417,"1441 EDT / 1841 UTC 8/23/2021\r\nHAVOC39, C-1...",8/23/2021,18:34,2021-08-23T18:34Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3418,1756 EDT / 2156 UTC 8/23/2021\r\nChicago OHare...,8/23/2021,21:53,2021-08-23T21:53Z,NaN,JATOC,UAS,NaN,ORD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3419,MA State PD observed a UAS over the airport at...,8/26/2021,0:47,2021-08-26T00:47Z,NaN,DEN,UAS,NaN,BOS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Export records to csv file
non_match.to_csv('Non-Standard-2536Remaining.csv', index = False)

In [8]:
non_std = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Non-Standard-2536Remaining.csv')
non_std

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
0,W-ROC reports aircraft reported a white UAS at...,1/4/2018,5:00,2018-01-04T05:00Z,ASA918,DEN,UAS,NaN,WOC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Reported a red and white UAS passed helicopter...,1/4/2018,8:31,2018-01-04T08:31Z,REH9,DEN,UAS,NaN,SAN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATCT personnel reported observing a large UAS ...,1/4/2018,10:50,2018-01-04T10:50Z,NaN,DEN,UAS,NaN,SHV,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Aircrew reported observing a light from a UAS,...",1/5/2018,11:35,2018-01-05T11:35Z,SKW3019,DEN,UAS,NaN,TUS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Reported observing a UAS operating over downto...,1/6/2018,9:13,2018-01-06T09:13Z,N31197,DEN,UAS,NaN,TUL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2531,"1335 EDT / 1735 UTC 8/23/2021\r\nHAVOC39, C130...",8/23/2021,17:30,2021-08-23T17:30Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2532,"1441 EDT / 1841 UTC 8/23/2021\r\nHAVOC39, C-1...",8/23/2021,18:34,2021-08-23T18:34Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,1756 EDT / 2156 UTC 8/23/2021\r\nChicago OHare...,8/23/2021,21:53,2021-08-23T21:53Z,NaN,JATOC,UAS,NaN,ORD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2534,MA State PD observed a UAS over the airport at...,8/26/2021,0:47,2021-08-26T00:47Z,NaN,DEN,UAS,NaN,BOS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Extract records that have runway headings
non_std_rwy = non_std[non_std['REMARKS'].str.contains('RWY', regex=False, case=False, na=False) | (non_std['REMARKS'].str.contains('RUNWAY', regex=False, case=False, na=False))] 
non_std_rwy  # WE have 479 records 

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
0,W-ROC reports aircraft reported a white UAS at...,1/4/2018,5:00,2018-01-04T05:00Z,ASA918,DEN,UAS,NaN,WOC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Aircrew reported observing a light from a UAS,...",1/5/2018,11:35,2018-01-05T11:35Z,SKW3019,DEN,UAS,NaN,TUS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Reported a red quad-copter UAS 2 feet in diame...,1/10/2018,5:06,2018-01-10T05:06Z,ALAMO03,NCRCC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,A pedestrian reported observing a UAS hovering...,1/11/2018,6:58,2018-01-11T06:58Z,NaN,DEN,UAS,NaN,PHX,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Observed a large UAS 50 feet off the left side...,1/12/2018,0:29,2018-01-12T00:29Z,BALL56,NCRCC,UAS,NaN,END,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528,"1647 EDT / 2047 UTC 8/21/2021\r\nRA13U, P28A, ...",8/21/2021,20:41,2021-08-21T20:41Z,NaN,JATOC,UAS,NaN,PNS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2530,2155 EDT / 0155 UTC 8/23/2021\r\nChicago OHare...,8/23/2021,1:51,2021-08-23T01:51Z,NaN,JATOC,UAS,NaN,ORD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,1756 EDT / 2156 UTC 8/23/2021\r\nChicago OHare...,8/23/2021,21:53,2021-08-23T21:53Z,NaN,JATOC,UAS,NaN,ORD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2534,MA State PD observed a UAS over the airport at...,8/26/2021,0:47,2021-08-26T00:47Z,NaN,DEN,UAS,NaN,BOS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Export these records to seperate csv file. Send to Lex 
non_std_rwy.to_csv('Non-Standard_Runways.csv', index = False)

In [11]:
# Extract the non-matching records to keep them in separate csv file for further investigation
non_std_rwy1 = non_std_rwy['REMARKS']
non_std_rwy1

0       W-ROC reports aircraft reported a white UAS at...
3       Aircrew reported observing a light from a UAS,...
7       Reported a red quad-copter UAS 2 feet in diame...
9       A pedestrian reported observing a UAS hovering...
10      Observed a large UAS 50 feet off the left side...
                              ...                        
2528    1647 EDT / 2047 UTC 8/21/2021\r\nRA13U, P28A, ...
2530    2155 EDT / 0155 UTC 8/23/2021\r\nChicago OHare...
2533    1756 EDT / 2156 UTC 8/23/2021\r\nChicago OHare...
2534    MA State PD observed a UAS over the airport at...
2535    2110 EDT / 0110 UTC  8/26/2021\r\nMASS State P...
Name: REMARKS, Length: 479, dtype: object

In [12]:
non_match = pd.merge(non_std, non_std_rwy1, how='outer',indicator=True, on ='REMARKS')
non_match

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,_merge
0,W-ROC reports aircraft reported a white UAS at...,1/4/2018,5:00,2018-01-04T05:00Z,ASA918,DEN,UAS,NaN,WOC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,Reported a red and white UAS passed helicopter...,1/4/2018,8:31,2018-01-04T08:31Z,REH9,DEN,UAS,NaN,SAN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,ATCT personnel reported observing a large UAS ...,1/4/2018,10:50,2018-01-04T10:50Z,NaN,DEN,UAS,NaN,SHV,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,"Aircrew reported observing a light from a UAS,...",1/5/2018,11:35,2018-01-05T11:35Z,SKW3019,DEN,UAS,NaN,TUS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,Reported observing a UAS operating over downto...,1/6/2018,9:13,2018-01-06T09:13Z,N31197,DEN,UAS,NaN,TUL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2533,"1335 EDT / 1735 UTC 8/23/2021\r\nHAVOC39, C130...",8/23/2021,17:30,2021-08-23T17:30Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2534,"1441 EDT / 1841 UTC 8/23/2021\r\nHAVOC39, C-1...",8/23/2021,18:34,2021-08-23T18:34Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2535,1756 EDT / 2156 UTC 8/23/2021\r\nChicago OHare...,8/23/2021,21:53,2021-08-23T21:53Z,NaN,JATOC,UAS,NaN,ORD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2536,MA State PD observed a UAS over the airport at...,8/26/2021,0:47,2021-08-26T00:47Z,NaN,DEN,UAS,NaN,BOS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [13]:
# remove the mathcing records. We have 2057 remaining record
non_match = non_match[(non_match._merge != 'both')]
non_match

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,_merge
1,Reported a red and white UAS passed helicopter...,1/4/2018,8:31,2018-01-04T08:31Z,REH9,DEN,UAS,NaN,SAN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,ATCT personnel reported observing a large UAS ...,1/4/2018,10:50,2018-01-04T10:50Z,NaN,DEN,UAS,NaN,SHV,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Reported observing a UAS operating over downto...,1/6/2018,9:13,2018-01-06T09:13Z,N31197,DEN,UAS,NaN,TUL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5,UAS crashed in the vicinity of Harvard Coliseum.,1/8/2018,4:41,2018-01-08T04:41Z,NaN,DEN,UAS,NaN,WOC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6,"Reported a 4-rotor UAS 10 NW of FLL at 2,500 f...",1/9/2018,21:09,2018-01-09T21:09Z,JBU1145,DEN,UAS,NaN,FLL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528,1555 Raleigh-Durham Airport PD observed a UAS ...,8/13/2021,16:03,2021-08-13T16:03Z,NaN,DEN,UAS,NaN,RDU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2529,Ground personnel observed a UAS with a red top...,8/17/2021,17:16,2021-08-17T17:16Z,NaN,DEN,UAS,NaN,LUF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2531,The Navy Yard advised that NSF Indianhead cont...,8/22/2021,13:39,2021-08-22T13:39Z,NaN,NCRCC,UAS,NaN,NCRCC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2533,"1335 EDT / 1735 UTC 8/23/2021\r\nHAVOC39, C130...",8/23/2021,17:30,2021-08-23T17:30Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [14]:
# Drop unnecessary columns
non_match = non_match.drop(['_merge'], axis=1)
non_match

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,NEARESTAIRPORT.CEDAR,AIRSPACECLASS.CEDAR,ACLOCATION.CEDAR,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR
1,Reported a red and white UAS passed helicopter...,1/4/2018,8:31,2018-01-04T08:31Z,REH9,DEN,UAS,NaN,SAN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATCT personnel reported observing a large UAS ...,1/4/2018,10:50,2018-01-04T10:50Z,NaN,DEN,UAS,NaN,SHV,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Reported observing a UAS operating over downto...,1/6/2018,9:13,2018-01-06T09:13Z,N31197,DEN,UAS,NaN,TUL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,UAS crashed in the vicinity of Harvard Coliseum.,1/8/2018,4:41,2018-01-08T04:41Z,NaN,DEN,UAS,NaN,WOC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Reported a 4-rotor UAS 10 NW of FLL at 2,500 f...",1/9/2018,21:09,2018-01-09T21:09Z,JBU1145,DEN,UAS,NaN,FLL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528,1555 Raleigh-Durham Airport PD observed a UAS ...,8/13/2021,16:03,2021-08-13T16:03Z,NaN,DEN,UAS,NaN,RDU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2529,Ground personnel observed a UAS with a red top...,8/17/2021,17:16,2021-08-17T17:16Z,NaN,DEN,UAS,NaN,LUF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2531,The Navy Yard advised that NSF Indianhead cont...,8/22/2021,13:39,2021-08-22T13:39Z,NaN,NCRCC,UAS,NaN,NCRCC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,"1335 EDT / 1735 UTC 8/23/2021\r\nHAVOC39, C130...",8/23/2021,17:30,2021-08-23T17:30Z,NaN,JATOC,UAS,NaN,SAT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# Export remaining 2057 records to csv file. Send to Lex to run her R code on these records
non_match.to_csv('Non-Standard_2057Remaining.csv', index = False)